## Transform Customer Data
1. Remove records with NULL customer_id 
1. Remove exact duplicate records
1. Remove duplicate records based on created_timestamp
1. CAST the columns to the correct Data Type
1. Write transformed data to the Silver schema

In [0]:
-- 1.

SELECT * 
FROM gizmobox.bronze.v_customers
WHERE customer_id IS NOT NULL;

In [0]:
-- 2.

SELECT * 
FROM gizmobox.bronze.v_customers
WHERE customer_id IS NOT NULL
ORDER BY customer_id;

In [0]:
-- 3. 

SELECT DISTINCT *
FROM gizmobox.bronze.v_customers
WHERE customer_id IS NOT NULL
ORDER BY customer_id;

In [0]:
 
CREATE OR REPLACE TEMPORARY VIEW v_customers_distinct AS
SELECT DISTINCT *
FROM gizmobox.bronze.v_customers
WHERE customer_id IS NOT NULL;

In [0]:
select customer_id,
      MAX(created_timestamp)
FROM v_customers_distinct
GROUP BY customer_id

In [0]:
-- 3.

WITH cte_max AS
(
  select customer_id,
      MAX(created_timestamp) as max_created_timestamp
FROM v_customers_distinct
GROUP BY customer_id
)
SELECT t.*
from v_customers_distinct t
join cte_max m
on t.customer_id = m.customer_id
and t.created_timestamp = m.max_created_timestamp

In [0]:
-- 4.

WITH cte_max AS
(
  select customer_id,
      MAX(created_timestamp) as max_created_timestamp
FROM v_customers_distinct
GROUP BY customer_id
)
SELECT 
  cast(t.created_timestamp as timestamp) as created_timestamp,
  t.customer_id,
  t.customer_name,
  cast(t.date_of_birth as date) as date_of_birth,
  t.email,
  cast(t.member_since as date) as member_since,
  t.telephone
from v_customers_distinct t
join cte_max m
on t.customer_id = m.customer_id
and t.created_timestamp = m.max_created_timestamp

In [0]:
-- 5. 
CREATE TABLE gizmobox.silver.customers
AS

WITH cte_max AS
(
  select customer_id,
      MAX(created_timestamp) as max_created_timestamp
FROM v_customers_distinct
GROUP BY customer_id
)
SELECT 
  cast(t.created_timestamp as timestamp) as created_timestamp,
  t.customer_id,
  t.customer_name,
  cast(t.date_of_birth as date) as date_of_birth,
  t.email,
  cast(t.member_since as date) as member_since,
  t.telephone
from v_customers_distinct t
join cte_max m
on t.customer_id = m.customer_id
and t.created_timestamp = m.max_created_timestamp

In [0]:
select * from gizmobox.silver.customers

In [0]:
describe extended gizmobox.silver.customers